In [1]:
!pip install llama_index
!pip install langchain

In [2]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex

In [3]:
from llama_index.llms import openai
import json

class Chatbot:
    def __init__(self, api_key, query_engine):
        self.query_engine = query_engine
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        response = query_engine.query(user_input)

        message = {"role": "assistant", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message
    
    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)

In [4]:
# load data and set up query engine
# You need to specify the path where your source articles are in the next line of code, it needs to be the entire path to the files

documents = SimpleDirectoryReader('./directory where your help center articles are').load_data()
index = VectorStoreIndex(documents)
query_engine = index.as_query_engine()

In [ ]:
# Swap out your index below for whatever knowledge base you want
# You'll need to input your OPENAI API Key in the next line of code
# You can get your API Key with an OPENAI account

bot = Chatbot("Your OPENAI API Key", query_engine=query_engine)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    print(f"Bot: {response['content']}")